In [1]:
import os, numpy as np, pandas, sklearn
import mido, rtmidi, rtmidi_
np.random.seed(333)

In [2]:
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 '__init__.py',
 '__pycache__',
 'config.py',
 'data',
 'errors.py',
 'main.py',
 'ml-autoencoder.ipynb',
 'ml-v1.ipynb',
 'models.py',
 'rtmidi_',
 'sparql.py',
 'test_midi.py',
 'utils']

In [3]:
## NN libs
from sklearn.decomposition import PCA
import keras
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, Dropout, Flatten
from keras.layers import Conv2DTranspose, Reshape, MaxPooling2D, UpSampling2D, UpSampling1D, MaxPooling1D
from keras.layers import LocallyConnected1D, LocallyConnected2D
from keras.models import Model
from keras.callbacks import TensorBoard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# local libs
import config
from data import data, midi
from utils import io

 Context :: namedtuple(
[ max_t = float
, dt = float
, n_instances = int
, note_length = int
, bpm = float
, tempo = float
, ticks_per_beat = int
]



In [5]:
n: int = 4
context, x_train = data.init(n)
print(x_train.shape)

Setting up params

 >> Context(max_t=10.0, dt=0.01, n_instances=1000, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
Importing midi-data

[INFO] :
 |  reading file: ../datasets/examples/01 16th Snare copy.mid
[INFO] :
 |  reading file: ../datasets/examples/01 16th Snare.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Cym copy.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Cym.mid

Encoding midi-data
 [<midi file '../datasets/examples/01 16th Snare copy.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 16th Snare.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 8th Cym copy.mid' type 0, 1 tracks, 68 messages>, <midi file '../datasets/examples/01 8th Cym.mid' type 0, 1 tracks, 68 messages>]
(4, 1000, 127)


# Autoencoder
Input -> hidden layer -> output (=input)
Hidden layer has few dimension. Latent space of this layer should produce automatic categorization.

In [6]:
y_train = x_train
n_samples = x_train[0]
input_shape = x_train.shape[1:] # shape of a single sample
output_shape = y_train.shape[1:]
# output_length = y_train.shape[1]
# output_length = (y_train[0]).shape[0] # = length of an individual label
x_train.shape

(4, 1000, 127)

In [7]:
y_train.shape

(4, 1000, 127)

In [8]:
output_shape

(1000, 127)

In [47]:
def encoder(input_shape, output_shape, dropout=0.10):
    t = 1000
    input_layer = Input(shape=input_shape)
    x = input_layer
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    shape = (10,100) # 1 additional dimension
    x = Dense(np.prod(shape), activation='relu')(x) # 4*4*8 = 128
    x = Reshape(shape)(x)
    x = UpSampling1D(10)(x)
    x = Conv1D(100, 2, strides=2, activation='relu')(x) # 50,100
    x = UpSampling1D(output_shape[0] / 50)(x)
    x = Dense(output_shape[1], activation='relu')(x) # 
    # x = Dense(output_length, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=x)
    #     model.add(Dropout(dropout))
    return model, model.summary

dropout = 0.
model, summary = encoder(input_shape, output_shape, dropout)
summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 1000, 127)         0         
_________________________________________________________________
flatten_32 (Flatten)         (None, 127000)            0         
_________________________________________________________________
dense_66 (Dense)             (None, 100)               12700100  
_________________________________________________________________
dense_67 (Dense)             (None, 1000)              101000    
_________________________________________________________________
reshape_32 (Reshape)         (None, 10, 100)           0         
_________________________________________________________________
up_sampling1d_33 (UpSampling (None, 100, 100)          0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 50, 100)           20100     
__________

In [51]:
learning_rate = 0.001
# sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
optimizer = Adam(lr=learning_rate)
# top_k_categorical_accuracy(y_true, y_pred, k=5)
# https://keras.io/metrics/
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy','mean_squared_error'])

In [52]:
batch_size = 1
# n epochs = n iterations over all the training data
epochs = 9

In [53]:
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

Train on 3 samples, validate on 1 samples
Epoch 1/9
3/3 [==============================] - 2s 532ms/step - loss: 2.5627 - acc: 0.1773 - mean_squared_error: 344081.7969 - val_loss: 18.1346 - val_acc: 0.2000 - val_mean_squared_error: 527711.2500
Epoch 2/9
3/3 [==============================] - 1s 263ms/step - loss: 2.5615 - acc: 0.1390 - mean_squared_error: 351183.1514 - val_loss: 18.1726 - val_acc: 0.1000 - val_mean_squared_error: 534039.8750
Epoch 3/9
3/3 [==============================] - 1s 253ms/step - loss: 2.5612 - acc: 0.0390 - mean_squared_error: 355507.1465 - val_loss: 18.2189 - val_acc: 0.0000e+00 - val_mean_squared_error: 539033.7500
Epoch 4/9
3/3 [==============================] - 1s 248ms/step - loss: 2.5602 - acc: 0.2057 - mean_squared_error: 359108.9001 - val_loss: 18.2169 - val_acc: 0.3000 - val_mean_squared_error: 543176.1875
Epoch 5/9
3/3 [==============================] - 1s 255ms/step - loss: 2.5600 - acc: 0.2390 - mean_squared_error: 361118.1725 - val_loss: 18.1878 